# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846831415865                   -0.70    4.5
  2   -7.852316100793       -2.26       -1.53    1.0
  3   -7.852646225740       -3.48       -2.52    3.2
  4   -7.852646677568       -6.35       -3.35    2.0
  5   -7.852646685845       -8.08       -4.76    1.2
  6   -7.852646686722       -9.06       -5.19    3.5
  7   -7.852646686729      -11.16       -6.00    1.2
  8   -7.852646686730      -12.38       -7.34    2.8
  9   -7.852646686730   +  -14.45       -7.50    2.2
 10   -7.852646686730      -14.45       -8.70    2.0
 11   -7.852646686730      -14.75       -9.51    2.2
 12   -7.852646686730   +    -Inf       -9.51    1.0
 13   -7.852646686730      -14.75      -10.85    2.0
 14   -7.852646686730   +    -Inf      -11.54    3.2
 15   -7.852646686730   +    -Inf      -12.31    2.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846837169981                   -0.70           4.8
  2   -7.852525332796       -2.25       -1.63   0.80    2.0
  3   -7.852636093671       -3.96       -2.71   0.80    1.0
  4   -7.852646400111       -4.99       -3.30   0.80    2.2
  5   -7.852646681803       -6.55       -4.10   0.80    1.8
  6   -7.852646686371       -8.34       -4.74   0.80    1.2
  7   -7.852646686722       -9.46       -5.73   0.80    1.5
  8   -7.852646686730      -11.13       -6.50   0.80    2.5
  9   -7.852646686730      -12.67       -7.26   0.80    1.8
 10   -7.852646686730      -14.05       -7.90   0.80    2.2
 11   -7.852646686730      -15.05       -9.62   0.80    1.0
 12   -7.852646686730   +    -Inf       -9.43   0.80    3.5
 13   -7.852646686730   +  -14.57      -10.22   0.80    1.0
 14   -7.852646686730      -14.75      -11.32   0.80    1.0
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.355734e+01     3.464656e+00
 * time: 0.4147779941558838
     1     9.410974e-01     1.728638e+00
 * time: 0.6247811317443848
     2    -1.540031e+00     2.209211e+00
 * time: 0.6499600410461426
     3    -3.771632e+00     1.737548e+00
 * time: 0.6862571239471436
     4    -5.105760e+00     1.704929e+00
 * time: 0.7221651077270508
     5    -6.733963e+00     8.397156e-01
 * time: 0.7570540904998779
     6    -6.734933e+00     9.495873e-01
 * time: 0.7811939716339111
     7    -7.358988e+00     5.405083e-01
 * time: 0.8061449527740479
     8    -7.500927e+00     4.166405e-01
 * time: 0.8302550315856934
     9    -7.613134e+00     7.772928e-02
 * time: 0.8541049957275391
    10    -7.665810e+00     1.264413e-01
 * time: 0.878046989440918
    11    -7.705636e+00     1.031527e-01
 * time: 0.9013900756835938
    12    -7.731463e+00     9.309048e-02
 * time: 0.924854040145874
    13    -7.765757e+00     1.100394e-01
 * time: 0.947920083999

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846902251541                   -0.70    4.8
  2   -7.852328385708       -2.27       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686715                   -2.55
  2   -7.852646686730      -10.82       -6.04
  3   -7.852646686730   +  -14.57      -12.74


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 6.692767082510792e-13
|ρ_newton - ρ_scfv| = 2.7704352639104724e-13
|ρ_newton - ρ_dm|   = 1.116437130759343e-9
